In [13]:
import pandas as pd
import json

In [14]:
price_history = pd.read_json('price_collection.json')
price_history.columns = ['Date', 'Ticker', 'Volume', 'Open', 'Low', 'High', 'Close']
price_history.sort_values('Date', ascending=True, inplace=True)

with open('lotsize.json', "r") as read_file:
    lotsize = json.load(read_file)
    read_file.close()

black_list = ('DSKY','LNZL','LNZLP', 'ARSA', 'ASSB', 'AVAN', 'BISVP', 'BRZL', 'BSPBP', 'CHGZ', 'DZRD', 'DZRDP', 'EELT', 'ELTZ', 'GEMA', 'IGST', 'IGSTP', 'JNOS', 'JNOSP', 'KBSB', 'KCHE', 'KCHEP', 'KGKC', 'KGKCP', 'KMEZ', 'KOGK', 'KRKN', 'KROTP', 'KRSB', 'KRSBP', 'KTSB', 'KTSBP', 'KUBE', 'KUZB', 'LPSB', 'LVHK', 'MAGE', 'MAGEP', 'MFGS', 'MFGSP', 'MGNZ', 'MGTS', 'MISB', 'MISB', 'MISBP', 'MRKK', 'MRSB', 'NFAZ', 'NKSH', 'NNSB', 'NNSBP', 'OMZZP', 'PRMB', 'RBCM', 'RDRB', 'RGSS', 'ROSB', 'RTGZ', 'RTSB', 'RTSBP', 'RUSI', 'RZSB', 'SAGO', 'SAGOP', 'SARE', 'SAREP', 'SIBG', 'SLEN', 'STSB', 'STSBP', 'SVET', 'TASB', 'TASBP', 'TGKDP', 'TNSE', 'TORS', 'TORSP', 'TUZA', 'UCSS', 'UKUZ', 'URKZ', 'USBN', 'UTAR', 'VGSB', 'VGSBP', 'VJGZ', 'VJGZP', 'VLHZ', 'VRSBP', 'WTCM', 'WTCMP', 'YKEN', 'YKENP', 'YRSB', 'YRSBP', 'ZILL')

In [15]:
#prevent SettingWithCopyWarning message from appearing
pd.options.mode.chained_assignment = None
ranking = price_history.loc[price_history['Date'] == '2023-05-25']
ranking['Avg_sum'] = ranking['Volume'] * (ranking['High']+ranking['Low']+ranking['Close']) / 3
ranking.sort_values('Avg_sum', ascending=True, inplace=True)
ranking['DecileRank']= pd.qcut(ranking['Avg_sum'], q = 10, labels = False)

In [16]:
#FMBA ver3
idea_end = []
idea_active = []
idea_ready = []
lag_price = 1.005
ema_days = 20
for share in price_history.Ticker.unique():
    liquid = False
    ################
    #select data for each ticker
    period = price_history[(price_history['Ticker'] == share) & (price_history['Date'] > '2022-11-01')]

    #searching high volume over 10 ema
    #create a short-list to count ema10 volume
    volume_check = period['Volume'].to_frame()
    volume_check['Volume_EMA10'] = volume_check['Volume'].ewm(span=10).mean()
    volume_check['increase'] = volume_check['Volume'] / volume_check['Volume_EMA10']

    #find all pump days high-volume min x2 over ema10 at green day mim +8% price
    volume_check = volume_check [10:]
    pump_day = []
    for indx in volume_check.index:
        if volume_check.at[indx, 'increase'] > 2:
            if period.at[indx, 'Close'] > period.at[indx, 'Open'] * 1.08:
                pump_day.append(indx)
    pump_day.sort

    #counting emas and ATR
    if len(pump_day) > 0:        
        ema_atr = period['Close'].to_frame()
        ema_atr['price_EMA'] = ema_atr['Close'].ewm(span=ema_days).mean()
        #counting ATR
        for i,tek in enumerate(ema_atr.index):
            if i == 0:
                ema_atr.at[tek,'TR'] = period.at[tek,'High'] - period.at[tek,'Low']
            else:
                pred = ema_atr.index[i-1]
                tr1 = period.at[tek,'High'] - period.at[tek,'Low']
                tr2 = abs(period.at[tek,'High'] - period.at[pred,'Close'])
                tr3 = abs(period.at[pred,'Close'] - period.at[tek,'Low'])
                ema_atr.at[tek,'TR'] = max(tr1, tr2, tr3)
        ema_atr['ATR'] = ema_atr['TR'].ewm(alpha=1/14).mean()

        #check strategy    
        share_active = False
        to_buy =  False
        prev_stop_loss = 0
        take_profit = 0
        stop_loss = 0
        for i,indx in enumerate(ema_atr.index):
            if indx in pump_day and not share_active:
                to_buy = True
                continue
            prev = ema_atr.index[i-1]
            if share_active:
                take_profit = period.at[prev, 'Close'] + 1.5 * ema_atr.at[prev, 'ATR']                
                stop_loss = period.at[prev, 'Close'] - 1 * ema_atr.at[prev, 'ATR']
                stop_loss = max(stop_loss, prev_stop_loss)
                prev_stop_loss = stop_loss
                if period.at[indx, 'High'] >= take_profit:
                    share_active = False
                    idea_res = round((take_profit/buy_price-1)*100, 2)
                    idea_end.append([share, buy_date, period.at[indx, 'Date'], idea_res, liquid])            
                elif period.at[indx, 'Low'] <= stop_loss:
                    share_active = False
                    idea_res = round((stop_loss/buy_price-1)*100, 2)
                    idea_end.append([share, buy_date, period.at[indx, 'Date'], idea_res, liquid])            
            else:
                if to_buy:
                    if (period.at[indx, 'Low'] <= ema_atr.at[indx, 'price_EMA'] * lag_price ):
                        lot_trade = volume_check.at[indx, 'Volume_EMA10'] / lotsize.get(share)                    
                        if lot_trade > 10000:
                            liquid = True                        
                        buy_price = ema_atr.at[indx, 'price_EMA'] * lag_price
                        buy_date = period.at[indx, 'Date']
                        share_active = True
                        to_buy = False
        if share_active:
            idea_res = round((ema_atr.at[indx, 'Close']/buy_price-1)*100, 2)
            idea_active.append([buy_date, share, idea_res, round(buy_price,5), round(take_profit,5), round(stop_loss,5), ranking.at[indx,'DecileRank'], liquid])
        else:
            if to_buy:
                lot_trade = volume_check.at[indx, 'Volume_EMA10'] / lotsize.get(share)                
                if lot_trade > 10000:
                    liquid = True
                buy_price = ema_atr.at[indx, 'price_EMA'] + (2/(ema_days+1))*(period.at[indx,'Close']-ema_atr.at[indx, 'price_EMA'])
                idea_ready.append([share, round(buy_price * lag_price , 5), ranking.at[indx,'DecileRank'],liquid])

In [17]:
mas = [0.049, 0.099, 0.124, 0.141, 0.165, 0.199, 0.249, 0.332, 0.499, 0.999]
for posi in mas:
    res = 1000000
    depo = res
    depo_fact = res
    active_posi = {}
    period = price_history[price_history['Date'] > '2022-11-01']
    for d in period.Date.unique():
        for share in idea_end:        
            if share[2] == d:
                tick_posi = active_posi.pop(share[0],0)            
                depo += tick_posi * share[3]/100
                depo_fact += tick_posi * (1+share[3]/100)
            elif share[1] == d and share[0] not in black_list:
                tick_posi = posi * depo
                if tick_posi <= depo_fact:
                    depo_fact -= tick_posi
                    active_posi[share[0]] = tick_posi
        if posi == 0.49:
            print(active_posi)
    print(posi, round(depo_fact,2), round(depo_fact/res,2))    

0.049 1301818.17 1.3
0.099 1550409.72 1.55
0.124 1698173.35 1.7
0.141 1757524.3 1.76
0.165 1869132.51 1.87
0.199 2138784.61 2.14
0.249 2316979.44 2.32
0.332 2468439.85 2.47
0.499 2108578.05 2.11
0.999 2220971.35 2.22
